In [6]:
import json
import time
import requests
import sqlite3
import pandas as pd

##### Capitalisation boursière = Cours actuel x Offre en circulation

##### Désigne la valeur marchande totale de l’offre en circulation d’une crypto-monnaie. Semblable à la mesure du marché boursier  qui multiplie le cours par action par les actions facilement disponibles sur le marché (non détenues ni bloquées par des  initiés, des gouvernements).

In [7]:
## REQUEST
#Obtenir le prix actuel de toutes les crypto-monnaies dans toutes les autres devises prises en charge dont vous avez besoin.

response_5 = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=bitcoin%2Clitecoin&vs_currencies=eur%2Cusd&include_market_cap=true&include_24hr_vol=true&include_24hr_change=true&include_last_updated_at=true&precision=2")
simple_price = response_5.json()


#print(simple_price)
#print(simple_price["bitcoin"]["eur_market_cap"])


In [8]:
## PARSE

def parse_api_coingecko(simple_price):
    for dict in simple_price["bitcoin"]:
        eur = simple_price["bitcoin"]["eur"]
        eur_market_cap = simple_price["bitcoin"]["eur_market_cap"]
        eur_24h_vol = simple_price["bitcoin"]["eur_24h_vol"]
        eur_24h_change = simple_price["bitcoin"]["eur_24h_change"]
        usd = simple_price["bitcoin"]["usd"]
        usd_market_cap = simple_price["bitcoin"]["usd_market_cap"]
        usd_24h_vol = simple_price["bitcoin"]["usd_24h_vol"]
        usd_24h_change = simple_price["bitcoin"]["usd_24h_change"]
        last_updated_at = simple_price["bitcoin"]["last_updated_at"]
        print('euro:',eur,"\n"'dollar:',usd,"\n"'capitalisation boursiere €:',eur_market_cap,"\n"'Volume de négociation sur 24 h:',eur_24h_vol,"\n"'derniere Maj Prix:',last_updated_at)
        return eur, usd, eur_market_cap, eur_24h_vol




In [9]:

parse_api_coingecko(simple_price)


euro: 24036.23 
dollar: 25978.37 
capitalisation boursiere €: 467860561363.9191 
Volume de négociation sur 24 h: 13972244769.263506 
derniere Maj Prix: 1692786843


(24036.23, 25978.37, 467860561363.9191, 13972244769.263506)

In [ ]:
## BDD
#création de la Bdd SQLite si existe + création de la connexion
conn = sqlite3.connect("ProjectBTC.db")
cur = conn.cursor()

In [ ]:
## Bdd ajout colonne 
#req_1 = """ALTER TABLE btc ADD COLUMN eur_market_cap INTEGER"""
req_1 = """ALTER TABLE btc ADD COLUMN eur_24h_vol INTEGER"""
# Exécution de la requete 
cur.execute(req_1) 
# Envoyer la requete 
conn.commit() 
# Fermer la connexion 
conn.close()

In [ ]:
## BDD
# requete création table si n'existe pas
req = """CREATE TABLE IF NOT EXISTS btc(
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    eur INTEGER NOT NULL, 
    usd INTEGER NOT NULL,    
    last_update INTEGER NOT NULL,
    eur_market_cap INTEGER NOT NULL,
    eur_24h_vol INTEGER NOT NULL)""" 
# Exécution de la requete 
cur.execute(req) 
# Envoyer la requete 
conn.commit() 
# Fermer la connexion 
conn.close()

In [10]:
## Transformation = TIME
last_updated_1 = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(last_updated_at))
print(last_updated_1 )

NameError: name 'last_updated_at' is not defined

In [ ]:
## INSERT
conn = sqlite3.connect("ProjectBTC.db")
cur = conn.cursor()
req1 = 'INSERT INTO btc(eur,usd,last_update,eur_market_cap,eur_24h_vol) VALUES(?,?,?,?,?)'
val1 = (eur,usd,last_updated_1,eur_market_cap,eur_24h_vol)
cur.execute(req1,val1)
conn.commit() 
cur.close()

In [ ]:
## UPDATE unixepoch line

#conn = sqlite3.connect("ProjectBTC.db")
#cur = connexion.cursor()
#req2 = 'update btc set last_update = datetime(last_update, 'unixepoch', 'localtime') where last_update like '16%';'

#cur.execute(req2)

#cur.close()

In [ ]:
## PRINT
cur = conn.cursor()
# cur.execute("select * from btc")
cur.execute("select eur,usd,last_update,eur_market_cap,eur_24h_vol from btc order by last_update DESC limit(10)")
records = cur.fetchall()

print(records[0])

In [ ]:
#DATAFRAME

df_btc = pd.read_sql('select eur,usd,last_update,eur_market_cap,eur_24h_vol from btc order by last_update',conn,)
#df_btc = pd.read_sql('''select eur,usd,last_update from btc where last_update like '2023-08-17%' ''',conn,)
#df_btc = pd.read_sql('select eur,usd,last_update from btc order by last_update DESC limit(10)',conn,)
df_btc

In [ ]:
# DETECTION DE DOUBLON
df_btc_doublon = pd.read_sql('''SELECT COUNT(*) AS nbr_doublon, eur,last_update,eur_market_cap FROM btc  where last_update like '2023-08-05%' GROUP BY last_update HAVING COUNT(*) > 1 ''',conn,)
df_btc_doublon


In [ ]:
# SUPPRESSION DE DOUBLON => df_btc
df_btc_no_duplicated = df_btc.drop_duplicates()
df_btc_no_duplicated

In [ ]:
df_btc_no_duplicated.duplicated()

In [ ]:
print(df_btc_no_duplicated.duplicated().sum())

In [ ]:
# SUPPRESSION COLONNE 
#df_btc_0 = df_btc.drop(['last_update'],axis=1)
#df_btc_0

In [ ]:
## TRANSFORMATION DATAFRAME
#df_btc_1 = pd.to_datetime(df_btc['last_update'], format='ISO8601')
#df_btc_1

In [ ]:
## TRANSFORMATION DATAFRAME
# Convert datetype to string

#df_btc_1 = df_btc['last_update'].astype(str)

In [ ]:
## TRANSFORMATION DATAFRAME
# concatenation dataframe
#df_btc_01 = pd.concat([df_btc_0, df_btc_1],axis=1)
#df_btc_01

In [ ]:
## VISUALISATION

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

#%matplotlib inline

#plt.figure(figsize=(15,6))

#x = df_btc["last_update"]
#y = df_btc["eur"]  
      
#plt.plot(x, 
 #        y,
 #        linewidth = 4)

#plt.title('Evolution du prix du Bitcoin en €')
#plt.xlabel('Date')
#plt.xticks(rotation = 45)
#plt.ylabel('Prix du Bitcoin (en euros et en dollar')



In [ ]:
## VISUALISATION

#import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.cm as cm
#import plotly.graph_objects as go
#import plotly.express as px
#from plotly.subplots import make_subplots

#x = df_btc["last_update"]
#y = df_btc["eur"]  

#y1 = df_btc["usd"]

#fig1 = px.line(df_btc, x=x, y=y1)
#fig1.update_layout(legend_title_text = "Evolution du prix du Bitcoin en $")
#fig1.update_xaxes(title_text="Last Update")
#fig1.update_yaxes(title_text="btc dollar")

#fig.show()
#fig1.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=("BTC euro", "BTC dollar","Capitalisation boursière","volume d'échange eur 24h"))

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=1, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="euro", row=1, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="dollar", row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="eur_market_cap", row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Last update", row=2, col=1)
# Update yaxis properties
fig.update_yaxes(title_text="eur_24h_vol", row=2, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur"],
name="cours BTC euro"
), row=1, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["usd"],
name="cours BTC dollar"
), row=2, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur_market_cap"],
name="capitalisation boursière"
), row=3, col=1)

fig.append_trace(go.Scatter(
x = df_btc_no_duplicated["last_update"],
y = df_btc_no_duplicated["eur_24h_vol"],
name="Volume de négociation sur 24 h"
), row=4, col=1)

fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(height=1200, width=1000, title_text="Evolution du cours du Bitcoin", hovermode = "x unified" )
fig.show()